# **Correlación lineal entra variables climáticas**

## **Importación de librerías**



In [40]:
%%capture

# Instalar mapas en mosaico
!pip3 install contextily

# Extensión geoespacial para xarray
!pip install rioxarray

# Herramientas para scikit-learn con datos espaciales
!pip install pyimpute

# Proyecciones de mapas en matplotlib
!python -m pip install basemap

# Crear cuadrículas hexagonales
!pip install geohexgrid

# Barra de escala para matplotlib
!pip install matplotlib-scalebar

In [41]:
# Se importan funciones auxiliares desde GitHub requeridas para el desarrollo de
# este práctico
!git clone https://github.com/jsblandon/weather_data_uy_preprocessing.git
import sys
sys.path.append('/content/weather_data_uy_preprocessing')

fatal: destination path 'weather_data_uy_preprocessing' already exists and is not an empty directory.


In [42]:
import matplotlib.pyplot as plt
import contextily as cx                                                         # Mapas base
import geopandas as gpd                                                         # Manipulación de datos espaciales
import numpy as np                                                              # Arrays y operaciones matemáticas
import pandas as pd                                                             # Manipulación y análisis de datos
import seaborn as sns                                                           # Gráficos estadísticos
import rasterio as rio                                                          # Lectura/escritura de datos raster geoespaciales
import rioxarray as rxr                                                         # Extiende xarray y rasterio
from shapely.geometry import mapping, Point, Polygon                            # Creación y manipulación de geometrías
import geohexgrid as ghg                                                        # Cuadrículas hexagonales
import xarray as xr                                                             # Arrays multidimensionales etiquetados
from matplotlib_scalebar.scalebar import ScaleBar                               # Barra de escala en gráficos
from geopandas import GeoDataFrame                                              # Creación de GeoDataFrames
from google.colab import files, drive                                           # Manejo de archivos en Google Colab
from matplotlib.axes._axes import _log as matplotlib_axes_logger                # Control de logs en ejes de Matplotlib
from sklearn.experimental import enable_iterative_imputer                       # Habilita Imputadores iterativos
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer          # Métodos de imputación
from sklearn.preprocessing import StandardScaler                                # Estandarización de datos
from shapely.geometry import mapping, Point, Polygon                            # Manejo de geometrías para recortes raster
from pyimpute import load_training_vector                                       # Carga vectores de entrenamiento geoespaciales
from weather_data_preprocessing import null_report                              # Preprocesamiento de datos meteorológicos

%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

sns.set_style('whitegrid')

In [43]:
drive.mount('/content/drive')                   # Acceso a archivos de Google Drive en Colab

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Carga de funciones

In [44]:
def generate_hex_ab_counts(gdf_av : GeoDataFrame, gdf_hex : GeoDataFrame, var : str )-> GeoDataFrame:
    f"""Funcion para sumar abundancias por hexagonos

    Recibe
    ------
        gdf_av : GeoDataFrame
            Debe tener la geometría, un identificador de la celda y la columna
            de a sumar

        gdf_hex : GeoDataFrame
            Contiene los polígonos de los hexagonos

        var : string
            Variable sobre la cual se desea hacer el conteo

    Devuelve
    --------
        gdf : GeoDataFrame
            Entrega también un geodataframe con los mismos campos pero para todas
            las celdas de la malla hexagonal

        Fuente: J.S. Blandon (2024).
    """
    gdf = gdf_av.groupby(['geometry','cell_id'])[var].sum().reset_index()
    gdf = pd.merge(gdf_hex, gdf, how= "outer", on= "cell_id")
    gdf = GeoDataFrame(gdf, crs= '5382', geometry= gdf['geometry_x'])
    gdf.drop(['geometry_x','geometry_y'],axis= 1, inplace= True)
    # gdf[var] = np.log1p(gdf[var].fillna(0))

    return gdf

In [45]:
def data_imputation(gdf_av : GeoDataFrame, ruta_raster : list, crs_raster : str, etiqueta : str, var_list : list)-> GeoDataFrame:
    f"""Funcion para prepararacion de variables exogenas para alimentar SDMs

    Recibe
    ------
        gdf_av : GeoDataFrame
            Debe tener la geometría y la columna de las etiquetas (presencia/ausencia)

        ruta_raster : list
            Lista con las rutas o única ruta de los rasters a utilizar

        crs_raster : str
            CRS del raster

        etiqueta : string
            Variable que identifica las presencias/ausencias

        var_list : list
            Lista de nombres para asignarle a las variables imputadas

    Devuelve
    --------
        gdf : GeoDataFrame
            Entrega un GeoDataFrame con las variables exogenas indexadas

        Fuente: J.S. Blandon (2024).
    """

    # Se hace una copia del GeoDataFrame de malla hexagonal que contiene los datos
    gdf_av_copy = gdf_av.copy()

    # Se reemplaza la geometría por el centroide de los hexagonos
    gdf_av_copy['geometry'] = gdf_av_copy['geometry'].centroid

    # Se reproyecta el GeoDataframe a la capa raster
    gdf_av_copy = gdf_av_copy.to_crs(crs_raster)

    # Se carga el vector
    var_vals, _ = load_training_vector(gdf_av_copy,
                                       ruta_raster,
                                       response_field=etiqueta)

    # Se indexan el raster imputado al GeoDataFrame
    if len(var_list) == 1:
        gdf_av_copy[var_list[0]] = var_vals.reshape(-1,)

    else:
        for i in range(len(var_list)):
            gdf_av_copy[var_list[i]] = var_vals[:,i].reshape(-1,)


    return gdf_av_copy

In [46]:
# Definición de ruta para almacenar figuras y/o datos para generarlas
ruta_figs = '/content/drive/MyDrive/Proyecto aplicado /figuras_documento_ob2/'

# **Alineación de datos raster: En peligro crítico (CR)**

### Carga de datos

In [ ]:
# Ruta de los datos de avistamientos
path = "/content/drive/MyDrive/Proyecto aplicado /Procesamiento variables exogenas/filtered_df_final2_CR.csv"

In [ ]:
# Carga de datos de avistamientos
df_cr =  pd.read_csv(path,parse_dates=["last_edited_date","observation_date"])

In [ ]:
# Variables clasificadas por el tipo de dato equivalente entre Python y R

char_attributes = ["checklist_id","country","country_code","state","state_code",
                   "locality","locality_id","locality_type","observer_id",
                   "sampling_event_identifier","protocol_type","protocol_code",
                   "project_code","group_identifier","trip_comments","scientific_name",
                   "time_observations_started"]

float_attributes= ["latitude","longitude","effort_distance_km",
                   "effort_hours","effort_speed_kmph","hours_of_day","year",
                   "day_of_year"]

int_attributes  = ["duration_minutes","number_observers","observation_count"]

bool_attributes = ["all_species_reported","species_observed"]

In [ ]:
# Conversion a tipos de datos string (cadenas)
df_cr[char_attributes] = df_cr[char_attributes].astype(str)

# Conversion a tipos de datos float (flotantes)
df_cr[float_attributes] = df_cr[float_attributes].astype(float)

# Conversion a tipos de datos integer (enteros)
df_cr[int_attributes] = df_cr[int_attributes].astype(int)

# Conversion a tipos de datos boolean (booleanos)
df_cr[bool_attributes] = df_cr[bool_attributes].astype(bool)

In [ ]:
# De DataFrame a GeoDataFrame
geometry = [Point(xy) for xy in zip(df_cr['longitude'], df_cr['latitude'])]
gdf_df_cr = GeoDataFrame(df_cr, crs='WGS84', geometry=geometry)

In [ ]:
# Selección de Variables de interés a partir del GeoDataFrame de avistamientos
gdf_df_cr = gdf_df_cr[["scientific_name", "year", "latitude","longitude",
                       "observation_count", "geometry","effort_hours",'effort_speed_kmph',
                       'effort_distance_km', 'number_observers']]

In [ ]:
# Se transforman los datos a una representacion basada en Unidades Estándar
gdf_df_cr = gdf_df_cr.to_crs(epsg = 5382)

### Shape de Departamentos de Colombia

In [ ]:
# Carga de Shape complementario Juan
# dir_Drive_shp = "/content/drive/MyDrive/research/codes/Proyecto aplicado /Departamentos_Junio_2024_shp/Departamento.shp"

# Carga de Shape complementario Victoria
dir_Drive_shp = "/content/drive/MyDrive/Proyecto aplicado /Departamentos_Junio_2024_shp/Departamento.shp"

states = gpd.read_file(dir_Drive_shp, use_arrow=True)

# Conversion al Sistema de Referencia de Coordendas
states = states.to_crs(crs=gdf_df_cr.crs)

# Region de Interés
roi = states[states['DeNombre'] == 'Risaralda']

# Reproyección
roi = roi.to_crs(crs=gdf_df_cr.crs)

### Variables exógenas: Elevación

In [ ]:
# Ruta de los datos Juan
# ruta_var_exogenas = "/content/drive/MyDrive/research/codes/Proyecto aplicado /Variables Bioclimaticas/"

# Ruta de los datos Victoria
ruta_var_exogenas = "/content/drive/MyDrive/Proyecto aplicado /Variables Bioclimaticas/"

# CRS de los rasters de WorldClim
wc21_crs = rio.open(ruta_var_exogenas + "Elevation_Output_ris/elevation_1.tif").crs

### Variables exógenas: Mapbiomas

In [ ]:
# Carga del raster
mapbiomas_ris = rxr.open_rasterio(ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2014.tif", masked = True).squeeze()
mb_crs = rio.open(ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2014.tif").crs

### Código principal

In [ ]:
roi.crs

<Projected CRS: EPSG:5382>
Name: SIRGAS-ROU98 / UTM zone 21S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Uruguay - west of 54°W, onshore and offshore.
- bounds: (-58.49, -36.63, -54.0, -30.09)
Coordinate Operation:
- name: UTM zone 21S
- method: Transverse Mercator
Datum: SIRGAS-ROU98
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
# Se grafican los datos
fig, axs = plt.subplots(1, 1, figsize=(10, 10), sharey=True, sharex=True)

gdf_df_cr.plot(ax=axs,c='b',markersize=8)
roi.plot(ax=axs,facecolor='none',edgecolor='k')
cx.add_basemap(ax=axs,crs=gdf_df_cr.crs,attribution_size=3)

axs.add_artist(ScaleBar(1))

plt.title("Registros Risaralda Henicorhina negreti", fontsize=22)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

# Guardar la figura en formato PNG con calidad 600 dpi y fondo transparente
fig.savefig(ruta_figs + 'fig_19_distribucion_espacial.png', dpi=600, bbox_inches='tight', format='png', transparent=True)

# Mostrar la figura
plt.show()

# Cerrar la figura después de guardarla
plt.close(fig)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# A partir del shape de la Región de Interés se genera una malla hexagonal
roi_hexagrid = ghg.make_grid_from_gdf(roi, R=500)

In [ ]:
# Recorte del shape
roi_hexagrid = gpd.clip(roi_hexagrid,roi)

In [ ]:
# Gráfica de la representacion hexagonal
fig, axs = plt.subplots(1, 1, figsize=(10, 10), sharey=True, sharex=True)

roi_hexagrid.plot(ax=axs,facecolor='black',linewidth=1.5)
gdf_df_cr.plot(ax=axs,c='b',markersize=8)
cx.add_basemap(ax=axs,crs=roi_hexagrid.crs,attribution_size=3)

axs.add_artist(ScaleBar(1))

plt.suptitle("Representacion de malla hexagonal", fontsize=22, y=0.92)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

# Guardar la figura en formato PNG con calidad 600 dpi y fondo transparente
fig.savefig(ruta_figs + 'fig_19_malla.png', dpi=600, bbox_inches='tight', format='png', transparent=True)

# Mostrar la figura
plt.show()

# Cerrar la figura después de guardarla
plt.close(fig)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Se une la información de la malla hexagonal a los datos
malla_avist_cr = gpd.sjoin(roi_hexagrid, gdf_df_cr, how="inner", predicate="contains")

In [ ]:
malla_avist_cr

,cell_id,geometry,index_right,scientific_name,year,latitude,longitude,observation_count,effort_hours,effort_speed_kmph,effort_distance_km,number_observers
750,"-2121,24359","POLYGON ((-1591000 10547323.393, -1591250 1054...",17038,Henicorhina negreti,2018.0,4.701116,-75.521315,0,1.283333,1.253766,1.609,2
882,"-2118,24360","POLYGON ((-1589000 10548189.418, -1588750 1054...",16453,Henicorhina negreti,2018.0,4.703466,-75.500879,0,2.166667,0.923077,2.000,3
885,"-2115,24361","POLYGON ((-1586750 10548622.431, -1586500 1054...",9808,Henicorhina negreti,2018.0,4.707334,-75.487361,0,0.333333,0.900000,0.300,2
878,"-2122,24360","POLYGON ((-1592000 10548189.418, -1591750 1054...",1171,Henicorhina negreti,2020.0,4.699431,-75.529094,0,0.766667,1.200000,0.920,1
878,"-2122,24360","POLYGON ((-1592000 10548189.418, -1591750 1054...",17037,Henicorhina negreti,2018.0,4.699931,-75.530951,0,0.883333,2.551698,2.254,2
...,...,...,...,...,...,...,...,...,...,...,...,...
14499,"-2175,24573","POLYGON ((-1630750 10640421.124, -1631000 1063...",15695,Henicorhina negreti,2023.0,5.480582,-75.896344,0,1.966667,0.091525,0.180,3
14499,"-2175,24573","POLYGON ((-1630750 10640421.124, -1631000 1063...",14203,Henicorhina negreti,2023.0,5.480582,-75.896344,0,0.633333,0.584211,0.370,1
14499,"-2175,24573","POLYGON ((-1630750 10640421.124, -1631000 1063...",14202,Henicorhina negreti,2023.0,5.480582,-75.896344,0,1.433333,0.439535,0.630,1
14499,"-2175,24573","POLYGON ((-1630750 10640421.124, -1631000 1063...",14200,Henicorhina negreti,2023.0,5.480582,-75.896344,0,1.466667,0.600000,0.880,1


In [ ]:
# Conteo de variables de esfuerzo y avistamientos por hexagono
variables = ["effort_speed_kmph", 'effort_hours', 'effort_distance_km',
             'number_observers']

malla_avist_cr_ev = generate_hex_ab_counts(malla_avist_cr,roi_hexagrid,'observation_count')

for var in variables:
    gdf_temp = generate_hex_ab_counts(malla_avist_cr,roi_hexagrid,var)
    malla_avist_cr_ev = pd.merge(malla_avist_cr_ev, gdf_temp.drop(columns='geometry'),
                                    how= "outer", on= "cell_id")

In [ ]:
# Malla de avistamientos
malla_avist_cr_ev.fillna(0, inplace= True)

In [ ]:
# Se crea la columna de etiquetas
malla_avist_cr_ev['label'] = malla_avist_cr_ev['observation_count'].apply(lambda x: 1 if x > 0 else 0)

# A manera de visualización se prueba si se asignaron correctamente las etiquetas
malla_avist_cr_ev[['label','observation_count']][malla_avist_cr_ev['observation_count'] != 0]

,label,observation_count
3321,1,1.0
3335,1,14.0
3413,1,36.0
3467,1,4.0
3494,1,3.0
3646,1,11.0
4656,1,3.0
4842,1,2.0
5843,1,1.0
5953,1,9.0


In [ ]:
# Se aplica función personalizada para imputar espacialmente los datos a partir
# del GeoDataFrame de avistamiento

# Rutas de las variables exógenas: WorldClim + Elevación
rutas_vars_exogenas = [ruta_var_exogenas + "Elevation_Output_ris/elevation_1.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_1.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_2.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_3.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_4.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_5.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_6.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_7.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_8.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_9.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_10.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_11.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_12.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_13.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_14.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_15.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_16.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_17.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_18.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_19.tif"
                       ]

# Lista de nombres de las nuevas variables
var_list = ['elevation','anual_mean_t', 'mean_diurnal_range',
            'isothermality', 't_seasonality', 'max_t_warmest_month',
            'min_t_coldest_month', 't_annual_range', 'mean_t_wettest_q',
            'mean_t_driest_q','mean_t_warmest_q','mean_t_coldest_q','annual_p',
            'p_wettest_m','p_driest_m', 'p_seasonality', 'p_wettest_q',
            'p_driest_q', 'p_warmest_q', 'p_coldest_q']

# Se imputan las nuevas variables al GeoDataFrame
malla_avist_cr_ev = data_imputation(malla_avist_cr_ev,
        rutas_vars_exogenas,
        wc21_crs,
        'label',
        var_list)

In [ ]:
malla_avist_cr_ev

,cell_id,observation_count,geometry,effort_speed_kmph,effort_hours,effort_distance_km,number_observers,label,elevation,anual_mean_t,...,mean_t_warmest_q,mean_t_coldest_q,annual_p,p_wettest_m,p_driest_m,p_seasonality,p_wettest_q,p_driest_q,p_warmest_q,p_coldest_q
0,"-2098,24366",0.0,POINT (-75.3791 4.72945),0.0,0.0,0.0,0.0,0,4294.0,3.497833,...,3.758,3.292667,1204.0,154.0,42.0,33.198063,402.0,185.0,349.0,268.0
1,"-2098,24374",0.0,POINT (-75.3794 4.76105),0.0,0.0,0.0,0.0,0,4249.0,3.748,...,4.006,3.550667,1242.0,159.0,44.0,33.470444,404.0,186.0,373.0,279.0
2,"-2098,24376",0.0,POINT (-75.37983 4.76657),0.0,0.0,0.0,0.0,0,4249.0,3.748,...,4.006,3.550667,1242.0,159.0,44.0,33.470444,404.0,186.0,373.0,279.0
3,"-2098,24378",0.0,POINT (-75.37998 4.77227),0.0,0.0,0.0,0.0,0,4249.0,3.748,...,4.006,3.550667,1242.0,159.0,44.0,33.470444,404.0,186.0,373.0,279.0
4,"-2098,24380",0.0,POINT (-75.37935 4.78099),0.0,0.0,0.0,0.0,0,4249.0,3.748,...,4.006,3.550667,1242.0,159.0,44.0,33.470444,404.0,186.0,373.0,279.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6482,"-2224,24544",0.0,POINT (-76.20126 5.36429),0.0,0.0,0.0,0.0,0,435.0,25.054001,...,25.415333,24.608,3403.0,428.0,159.0,30.314732,1175.0,598.0,868.0,1175.0
6483,"-2224,24546",0.0,POINT (-76.20088 5.36786),0.0,0.0,0.0,0.0,0,435.0,25.054001,...,25.415333,24.608,3403.0,428.0,159.0,30.314732,1175.0,598.0,868.0,1175.0
6484,"-2225,24539",0.0,POINT (-76.20753 5.34678),0.0,0.0,0.0,0.0,0,435.0,25.054001,...,25.415333,24.608,3403.0,428.0,159.0,30.314732,1175.0,598.0,868.0,1175.0
6485,"-2225,24541",0.0,POINT (-76.20546 5.35125),0.0,0.0,0.0,0.0,0,435.0,25.054001,...,25.415333,24.608,3403.0,428.0,159.0,30.314732,1175.0,598.0,868.0,1175.0


In [ ]:
malla_avist_cr_ev.dropna(inplace=True)

In [ ]:
# Se aplica función personalizada para imputar espacialmente los datos a partir
# del GeoDataFrame de avistamiento

# Rutas de las variables exógenas: MapBiomas
rutas_vars_exogenas = [ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2014.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2015.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2016.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2017.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2018.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2019.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2020.tif"]

# Lista de nombres de las nuevas variables
var_list = ['lulc_2014','lulc_2015','lulc_2016','lulc_2017','lulc_2018','lulc_2019','lulc_2020']

# Se imputan las nuevas variables al GeoDataFrame
malla_avist_cr_ev = data_imputation(malla_avist_cr_ev,
        rutas_vars_exogenas,
        mb_crs,
        'label',
        var_list)

<ipython-input-6-8732c9022bfa>:33: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_av_copy['geometry'] = gdf_av_copy['geometry'].centroid


In [ ]:
# Se hace una copia del GeoDataFrame
malla_avist_cr_ev_mb = malla_avist_cr_ev.copy()

In [ ]:
# Se guardael GeoDataFrame para no tener que correr de nuevo el código anterior Juan
# malla_avist_cr_ev_mb.to_file('/content/drive/MyDrive/research/codes/Proyecto aplicado /datos_unidos_avist_mb_ris.shp')

# Se guardael GeoDataFrame para no tener que correr de nuevo el código anterior Victoria y Paula
malla_avist_cr_ev_mb.to_file('/content/drive/MyDrive/Proyecto aplicado /datos_unidos_avist_mb_ris.shp')

In [ ]:
# Estas columnas se descartan si ya tienen la información de estas variables en
# el GeoDataFrame de las variables de WorldClim + Elevacion
malla_avist_cr_ev_mb.drop(columns=['observation_count', 'effort_speed_kmph',
                                   'effort_hours','effort_distance_km',
                                   'number_observers', 'label'], inplace = True)

In [ ]:
# Se unen los datos de WorldClim + Elevacion con los de MapBiomas para el departamento
malla_avist_cr_ev_f = gpd.sjoin(malla_avist_cr_ev, malla_avist_cr_ev_mb,on_attribute='cell_id')

In [ ]:
# Se almacena el shapefile.

# malla_avist_cr_ev_f.to_file('/content/drive/MyDrive/research/codes/Proyecto aplicado /datos_unidos_avist_wc_mb_ris.shp')

# VICTORIA Y PAULA
malla_avist_cr_ev_f.to_file('/content/drive/MyDrive/Proyecto aplicado /datos_unidos_avist_wc_mb_ris.shp')


<ipython-input-43-b5d5528982f0>:7: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  malla_avist_cr_ev_f.to_file('/content/drive/MyDrive/Proyecto aplicado /datos_unidos_avist_wc_mb_ris.shp')
/usr/local/lib/python3.10/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'observation_count' to 'observatio'
  ogr_write(
/usr/local/lib/python3.10/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'effort_speed_kmph' to 'effort_spe'
  ogr_write(
/usr/local/lib/python3.10/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'effort_hours' to 'effort_hou'
  ogr_write(
/usr/local/lib/python3.10/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'effort_distance_km' to 'effort_dis'
  ogr_write(
/usr/local/lib/python3.10/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'number_observers' t

# **Alineación de datos raster: En peligro (EN)**

## Carga de datos

In [78]:
# Ruta de los datos de avistamientos
path = "/content/drive/MyDrive/Proyecto aplicado /Procesamiento variables exogenas/filtered_df_final2_EN.csv"

# Ruta de los datos de avistamientos: JUAN
# path = "/content/drive/MyDrive/research/codes/Proyecto aplicado /Procesamiento variables exogenas/filtered_df_final2_EN.csv"

In [79]:
# Carga de datos de avistamientos
df_en =  pd.read_csv(path,parse_dates=["last_edited_date","observation_date"])

In [80]:
df_en

,checklist_id,last_edited_date,country,country_code,state,state_code,locality,locality_id,locality_type,latitude,...,trip_comments,scientific_name,observation_count,species_observed,status_co,effort_hours,effort_speed_kmph,hours_of_day,year,day_of_year
0,S52977144,2023-10-24 07:14:28.845856,Colombia,CO,Risaralda,CO-RIS,Via La Florida -- La Suiza,L6672775,P,4.744578,...,NaN,Penelope perspicax,4,True,En peligro,1.266667,3.434211,6.783333,2019.0,51.0
1,S54895798,2023-10-24 07:21:35.964443,Colombia,CO,Risaralda,CO-RIS,Via La Florida -- La Suiza,L6672775,P,4.744578,...,NaN,Penelope perspicax,0,False,En peligro,1.000000,0.000000,11.000000,2019.0,101.0
2,S61672676,2023-10-24 07:51:59.264845,Colombia,CO,Risaralda,CO-RIS,Via La Florida -- La Suiza,L6672775,P,4.744578,...,NaN,Penelope perspicax,2,True,En peligro,1.266667,1.602632,5.783333,2019.0,327.0
3,S52411371,2019-02-05 14:34:52,Colombia,CO,Risaralda,CO-RIS,Av 30 de Agosto Calle 39,L6720653,P,4.813223,...,NaN,Penelope perspicax,0,False,En peligro,0.416667,1.920000,7.583333,2019.0,35.0
4,S55205250,2021-08-19 17:25:20.91149,Colombia,CO,Risaralda,CO-RIS,Casa La Lolita Vereda Tribuna- Consota,L6729274,P,4.772187,...,NaN,Penelope perspicax,0,False,En peligro,5.000000,0.000000,12.833333,2019.0,110.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19185,G7618167,2023-10-23 19:05:10.524796,Colombia,CO,Risaralda,CO-RIS,RN Cauquitá,L5797600,H,4.843038,...,NaN,Penelope perspicax,0,False,En peligro,2.083333,0.000000,16.116667,2021.0,362.0
19186,G7629007,2023-10-24 04:02:22.868324,Colombia,CO,Risaralda,CO-RIS,"Vía Florida - el Cedral, Filandia CO-Quindío 4...",L17260826,P,4.746485,...,NaN,Penelope perspicax,0,False,En peligro,1.000000,0.000000,6.616667,2022.0,1.0
19187,G7627463,2022-01-01 19:40:45.006052,Colombia,CO,Risaralda,CO-RIS,Sendero La Linea,L7465754,H,5.180619,...,NaN,Penelope perspicax,0,False,En peligro,2.450000,0.640816,14.950000,2022.0,1.0
19188,G7629957,2022-01-02 00:31:41.538246,Colombia,CO,Risaralda,CO-RIS,Apia (pueblo),L2292807,H,5.105204,...,NaN,Penelope perspicax,0,False,En peligro,0.050000,0.000000,0.466667,2022.0,2.0


In [81]:
# Variables clasificadas por el tipo de dato equivalente entre Python y R

char_attributes = ["checklist_id","country","country_code","state","state_code",
                   "locality","locality_id","locality_type","observer_id",
                   "sampling_event_identifier","protocol_type","protocol_code",
                   "project_code","group_identifier","trip_comments","scientific_name",
                   "time_observations_started"]

float_attributes= ["latitude","longitude","effort_distance_km",
                   "effort_hours","effort_speed_kmph","hours_of_day","year",
                   "day_of_year"]

int_attributes  = ["duration_minutes","number_observers","observation_count"]

bool_attributes = ["all_species_reported","species_observed"]

In [82]:
# Conversion a tipos de datos string (cadenas)
df_en[char_attributes] = df_en[char_attributes].astype(str)

# Conversion a tipos de datos float (flotantes)
df_en[float_attributes] = df_en[float_attributes].astype(float)

# Conversion a tipos de datos integer (enteros)
df_en[int_attributes] = df_en[int_attributes].astype(int)

# Conversion a tipos de datos boolean (booleanos)
df_en[bool_attributes] = df_en[bool_attributes].astype(bool)

In [83]:
# De DataFrame a GeoDataFrame
geometry = [Point(xy) for xy in zip(df_en['longitude'], df_en['latitude'])]
gdf_df_en = GeoDataFrame(df_en, crs='WGS84', geometry=geometry)

In [84]:
# Selección de Variables de interés a partir del GeoDataFrame de avistamientos
gdf_df_en = gdf_df_en[["scientific_name", "year", "latitude","longitude",
                       "observation_count", "geometry","effort_hours",'effort_speed_kmph',
                       'effort_distance_km', 'number_observers']]

In [85]:
# Se transforman los datos a una representacion basada en Unidades Estándar
gdf_df_en = gdf_df_en.to_crs(epsg = 5382)

## Shape de Departamentos de Colombia

In [86]:
# # Carga de Shape complementario
dir_Drive_shp = "/content/drive/MyDrive/Proyecto aplicado /Departamentos_Junio_2024_shp/Departamento.shp"

# Carga de Shape complementario: JUAN
# dir_Drive_shp = "/content/drive/MyDrive/research/codes/Proyecto aplicado /Departamentos_Junio_2024_shp/Departamento.shp"

states = gpd.read_file(dir_Drive_shp, use_arrow=True)

# Conversion al Sistema de Referencia de Coordendas
states = states.to_crs(crs=gdf_df_en.crs)

# Region de Interés
roi = states[states['DeNombre'] == 'Risaralda']

# Reproyección
roi = roi.to_crs(crs=gdf_df_en.crs)

## Variables exógenas

In [87]:
# Ruta de los datos
ruta_var_exogenas = "/content/drive/MyDrive/Proyecto aplicado /Variables Bioclimaticas/"

# Ruta de los datos Juan
# ruta_var_exogenas = "/content/drive/MyDrive/research/codes/Proyecto aplicado /Variables Bioclimaticas/"

# CRS de los rasters de WorldClim
wc21_crs = rio.open(ruta_var_exogenas + "Elevation_Output_ris/elevation_1.tif").crs

In [88]:
# Carga del raster
mapbiomas_ris = rxr.open_rasterio(ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2014.tif", masked = True).squeeze()
mb_crs = rio.open(ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2014.tif").crs

## Código principal

In [89]:
# Se grafican los datos
fig, axs = plt.subplots(1, 1, figsize=(10, 10), sharey=True, sharex=True)

gdf_df_en.plot(ax=axs,c='b',markersize=8)
roi.plot(ax=axs,facecolor='none',edgecolor='k')
cx.add_basemap(ax=axs,crs=gdf_df_en.crs,attribution_size=3)

axs.add_artist(ScaleBar(1))

plt.title("Registros Risaralda Penelope perspicax", fontsize=22)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

# Guardar la figura en formato PNG con calidad 600 dpi y fondo transparente
fig.savefig(ruta_figs + 'fig_20_distribucion_espacial.png', dpi=600, bbox_inches='tight', format='png', transparent=True)

# Mostrar la figura
plt.show()

# Cerrar la figura después de guardarla
plt.close(fig)

Output hidden; open in https://colab.research.google.com to view.

In [90]:
# A partir del shape de la Región de Interés se genera una malla hexagonal
roi_hexagrid = ghg.make_grid_from_gdf(roi, R=500)

In [91]:
# Recorte del shape
roi_hexagrid = gpd.clip(roi_hexagrid,roi)

In [92]:
# Gráfica de la representacion hexagonal
fig, axs = plt.subplots(1, 1, figsize=(10, 10), sharey=True, sharex=True)

roi_hexagrid.plot(ax=axs,facecolor='black',linewidth=1.5)
gdf_df_en.plot(ax=axs,c='b',markersize=8)
cx.add_basemap(ax=axs,crs=roi_hexagrid.crs,attribution_size=3)

axs.add_artist(ScaleBar(1))

plt.suptitle("Representacion de malla hexagonal", fontsize=22, y=0.92)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

# Guardar la figura en formato PNG con calidad 600 dpi y fondo transparente
fig.savefig(ruta_figs + 'fig_20_malla.png', dpi=600, bbox_inches='tight', format='png', transparent=True)

# Mostrar la figura
plt.show()

# Cerrar la figura después de guardarla
plt.close(fig)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Se une la información de la malla hexagonal a los datos
malla_avist_en = gpd.sjoin(roi_hexagrid, gdf_df_en, how="inner", predicate="contains")

In [ ]:
malla_avist_en

,cell_id,geometry,index_right,scientific_name,year,latitude,longitude,observation_count,effort_hours,effort_speed_kmph,effort_distance_km,number_observers
750,"-2121,24359","POLYGON ((-1591000 10547323.393, -1591250 1054...",16997,Penelope perspicax,2018.0,4.701116,-75.521315,0,1.283333,1.253766,1.609,2
882,"-2118,24360","POLYGON ((-1589000 10548189.418, -1588750 1054...",16415,Penelope perspicax,2018.0,4.703466,-75.500879,0,2.166667,0.923077,2.000,3
885,"-2115,24361","POLYGON ((-1586750 10548622.431, -1586500 1054...",9782,Penelope perspicax,2018.0,4.707334,-75.487361,0,0.333333,0.900000,0.300,2
878,"-2122,24360","POLYGON ((-1592000 10548189.418, -1591750 1054...",1171,Penelope perspicax,2020.0,4.699431,-75.529094,0,0.766667,1.200000,0.920,1
878,"-2122,24360","POLYGON ((-1592000 10548189.418, -1591750 1054...",16996,Penelope perspicax,2018.0,4.699931,-75.530951,0,0.883333,2.551698,2.254,2
...,...,...,...,...,...,...,...,...,...,...,...,...
14499,"-2175,24573","POLYGON ((-1630750 10640421.124, -1631000 1063...",16618,Penelope perspicax,2018.0,5.480582,-75.896344,0,1.000000,1.000000,1.000,2
14499,"-2175,24573","POLYGON ((-1630750 10640421.124, -1631000 1063...",14169,Penelope perspicax,2023.0,5.480582,-75.896344,0,1.466667,0.600000,0.880,1
14499,"-2175,24573","POLYGON ((-1630750 10640421.124, -1631000 1063...",15664,Penelope perspicax,2023.0,5.480582,-75.896344,0,1.966667,0.091525,0.180,3
14499,"-2175,24573","POLYGON ((-1630750 10640421.124, -1631000 1063...",14171,Penelope perspicax,2023.0,5.480582,-75.896344,0,1.433333,0.439535,0.630,1


In [ ]:
# Conteo de variables de esfuerzo y avistamientos por hexagono
variables = ["effort_speed_kmph", 'effort_hours', 'effort_distance_km',
             'number_observers']

malla_avist_en_ev = generate_hex_ab_counts(malla_avist_en,roi_hexagrid,'observation_count')

for var in variables:
    gdf_temp = generate_hex_ab_counts(malla_avist_en,roi_hexagrid,var)
    malla_avist_en_ev = pd.merge(malla_avist_en_ev, gdf_temp.drop(columns='geometry'),
                                    how= "outer", on= "cell_id")

In [ ]:
# Malla de avistamientos
malla_avist_en_ev.fillna(0, inplace= True)

In [ ]:
# Se crea la columna de etiquetas
malla_avist_en_ev['label'] = malla_avist_en_ev['observation_count'].apply(lambda x: 1 if x > 0 else 0)

# A manera de visualización se prueba si se asignaron correctamente las etiquetas
malla_avist_en_ev[['label','observation_count']][malla_avist_en_ev['observation_count'] != 0]

,label,observation_count
369,1,15.0
404,1,4.0
439,1,8.0
450,1,2.0
506,1,6.0
...,...,...
1679,1,2.0
1791,1,12.0
1850,1,8.0
2978,1,6.0


In [ ]:
# Se aplica función personalizada para imputar espacialmente los datos a partir
# del GeoDataFrame de avistamiento

# Rutas de las variables exógenas: WorldClim + Elevación
rutas_vars_exogenas = [ruta_var_exogenas + "Elevation_Output_ris/elevation_1.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_1.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_2.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_3.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_4.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_5.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_6.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_7.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_8.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_9.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_10.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_11.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_12.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_13.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_14.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_15.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_16.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_17.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_18.tif",
                       ruta_var_exogenas + "WorldClim_Output_ris/worldclim_bio_19.tif"
                       ]

# Lista de nombres de las nuevas variables
var_list = ['elevation','anual_mean_t', 'mean_diurnal_range',
            'isothermality', 't_seasonality', 'max_t_warmest_month',
            'min_t_coldest_month', 't_annual_range', 'mean_t_wettest_q',
            'mean_t_driest_q','mean_t_warmest_q','mean_t_coldest_q','annual_p',
            'p_wettest_m','p_driest_m', 'p_seasonality', 'p_wettest_q',
            'p_driest_q', 'p_warmest_q', 'p_coldest_q']

# Se imputan las nuevas variables al GeoDataFrame
malla_avist_en_ev_wc = data_imputation(malla_avist_en_ev,
        rutas_vars_exogenas,
        wc21_crs,
        'label',
        var_list)

KeyboardInterrupt: 

In [ ]:
# Se descartan los valores faltantes y/o nulos
malla_avist_en_ev_wc.dropna(inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Se aplica función personalizada para imputar espacialmente los datos a partir
# del GeoDataFrame de avistamiento

# Rutas de las variables exógenas: MapBiomas
rutas_vars_exogenas = [ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2014.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2015.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2016.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2017.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2018.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2019.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ris/mapbiomas_ris_2020.tif"]

# Lista de nombres de las nuevas variables
var_list = ['lulc_2014','lulc_2015','lulc_2016','lulc_2017','lulc_2018','lulc_2019','lulc_2020']

# Se imputan las nuevas variables al GeoDataFrame
malla_avist_en_mb = data_imputation(malla_avist_en_ev,
        rutas_vars_exogenas,
        mb_crs,
        'label',
        var_list)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Le descartamos a este GeoDataGrame las variables de esfuerzo, el conteo de
# observaciones y las etiquetas ya que están en el otro GeoDataFrame al que vamos
# a unirlo
malla_avist_en_mb.drop(columns=["effort_speed_kmph", 'effort_hours', 'effort_distance_km',
             'number_observers', 'observation_count','label'], inplace=True)
malla_avist_en_mb

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,cell_id,geometry,lulc_2014,lulc_2015,lulc_2016,lulc_2017,lulc_2018,lulc_2019,lulc_2020
0,"-2098,24366",POINT (-75.3791 4.72945),29.0,29.0,29.0,13.0,13.0,13.0,13.0
1,"-2098,24374",POINT (-75.3794 4.76105),29.0,29.0,29.0,29.0,29.0,29.0,29.0
2,"-2098,24376",POINT (-75.37983 4.76657),13.0,13.0,13.0,13.0,13.0,13.0,13.0
3,"-2098,24378",POINT (-75.37998 4.77227),13.0,13.0,13.0,13.0,13.0,13.0,13.0
4,"-2098,24380",POINT (-75.37935 4.78099),13.0,13.0,29.0,13.0,13.0,13.0,13.0
...,...,...,...,...,...,...,...,...,...
6482,"-2224,24544",POINT (-76.20126 5.36429),3.0,3.0,3.0,3.0,3.0,3.0,3.0
6483,"-2224,24546",POINT (-76.20088 5.36786),3.0,3.0,3.0,3.0,3.0,3.0,3.0
6484,"-2225,24539",POINT (-76.20753 5.34678),3.0,3.0,3.0,3.0,3.0,3.0,3.0
6485,"-2225,24541",POINT (-76.20546 5.35125),3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [ ]:
# Se unen los datos de WorldClim + Elevacion con los de MapBiomas para el departamento
malla_avist_en_ev_f = gpd.sjoin(malla_avist_en_ev_wc, malla_avist_en_mb,on_attribute='cell_id')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
malla_avist_en_ev_f

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,cell_id,observation_count,geometry,effort_speed_kmph,effort_hours,effort_distance_km,number_observers,label,elevation,anual_mean_t,...,p_warmest_q,p_coldest_q,index_right,lulc_2014,lulc_2015,lulc_2016,lulc_2017,lulc_2018,lulc_2019,lulc_2020
0,"-2098,24366",0.0,POINT (-75.3791 4.72945),0.000,0.0,0.00,0.0,0,4294.0,3.497833,...,349.0,268.0,0,29.0,29.0,29.0,13.0,13.0,13.0,13.0
1,"-2098,24374",0.0,POINT (-75.3794 4.76105),0.000,0.0,0.00,0.0,0,4249.0,3.748,...,373.0,279.0,1,29.0,29.0,29.0,29.0,29.0,29.0,29.0
2,"-2098,24376",0.0,POINT (-75.37983 4.76657),0.000,0.0,0.00,0.0,0,4249.0,3.748,...,373.0,279.0,2,13.0,13.0,13.0,13.0,13.0,13.0,13.0
3,"-2098,24378",0.0,POINT (-75.37998 4.77227),0.000,0.0,0.00,0.0,0,4249.0,3.748,...,373.0,279.0,3,13.0,13.0,13.0,13.0,13.0,13.0,13.0
4,"-2098,24380",0.0,POINT (-75.37935 4.78099),0.000,0.0,0.00,0.0,0,4249.0,3.748,...,373.0,279.0,4,13.0,13.0,29.0,13.0,13.0,13.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6481,"-2224,24542",0.0,POINT (-76.20093 5.35628),1.575,6.4,4.13,15.0,0,435.0,25.054001,...,868.0,1175.0,6481,33.0,33.0,33.0,33.0,33.0,33.0,3.0
6482,"-2224,24544",0.0,POINT (-76.20126 5.36429),0.000,0.0,0.00,0.0,0,435.0,25.054001,...,868.0,1175.0,6482,3.0,3.0,3.0,3.0,3.0,3.0,3.0
6483,"-2224,24546",0.0,POINT (-76.20088 5.36786),0.000,0.0,0.00,0.0,0,435.0,25.054001,...,868.0,1175.0,6483,3.0,3.0,3.0,3.0,3.0,3.0,3.0
6484,"-2225,24539",0.0,POINT (-76.20753 5.34678),0.000,0.0,0.00,0.0,0,435.0,25.054001,...,868.0,1175.0,6484,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [ ]:
# Se almacena el shapefile.
malla_avist_en_ev_f.to_file('/content/drive/MyDrive/Proyecto aplicado /datos_unidos_avist_wc_mb_ris_en/datos_unidos_avist_wc_mb_ris_en.shp')

# Se almacena el shapefile Juan
# malla_avist_en_ev_f.to_file('/content/drive/MyDrive/research/codes/Proyecto aplicado /datos_unidos_avist_wc_mb_ris_en/datos_unidos_avist_wc_mb_ris_en.shp')

NameError: name 'malla_avist_en_ev_f' is not defined

# **Alineación de datos raster: Vulnerable(VU)**

## Carga de datos

In [62]:
# Ruta de los datos de avistamientos
path = "/content/drive/MyDrive/Proyecto aplicado /Procesamiento variables exogenas/filtered_df_final2_VU.csv"

# Ruta de los datos de avistamientos: JUAN
# path = "/content/drive/MyDrive/research/codes/Proyecto aplicado /Procesamiento variables exogenas/filtered_df_final2_EN.csv"

In [63]:
# Carga de datos de avistamientos
df_vu =  pd.read_csv(path,parse_dates=["last_edited_date","observation_date"])

<ipython-input-63-64fda2a4b7eb>:2: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vu =  pd.read_csv(path,parse_dates=["last_edited_date","observation_date"])


In [64]:
df_vu

,checklist_id,last_edited_date,country,country_code,state,state_code,locality,locality_id,locality_type,latitude,...,trip_comments,scientific_name,observation_count,species_observed,status_co,effort_hours,effort_speed_kmph,hours_of_day,year,day_of_year
0,S60360053,2023-11-25 21:52:57.837266,Colombia,CO,Antioquia,CO-ANT,"Urbanización Atalaya de San Jorge, Envigado, A...",L6664911,P,6.161704,...,NaN,Hypopyrrhus pyrohypogaster,0,False,Vulnerable,0.533333,0.750000,7.983333,2019.0,278.0
1,S61357410,2023-11-25 21:52:57.837266,Colombia,CO,Antioquia,CO-ANT,"Urbanización Atalaya de San Jorge, Envigado, A...",L6664911,P,6.161704,...,NaN,Hypopyrrhus pyrohypogaster,0,False,Vulnerable,0.650000,0.830769,7.483333,2019.0,315.0
2,S51579402,2023-11-25 21:52:57.837266,Colombia,CO,Antioquia,CO-ANT,"Urbanización Atalaya de San Jorge, Envigado, A...",L6664911,P,6.161704,...,NaN,Hypopyrrhus pyrohypogaster,0,False,Vulnerable,0.400000,0.000000,7.083333,2019.0,12.0
3,S59971734,2023-11-25 21:52:57.837266,Colombia,CO,Antioquia,CO-ANT,"Urbanización Atalaya de San Jorge, Envigado, A...",L6664911,P,6.161704,...,NaN,Hypopyrrhus pyrohypogaster,0,False,Vulnerable,0.750000,0.666667,7.700000,2019.0,264.0
4,S52070627,2023-11-25 21:52:57.837266,Colombia,CO,Antioquia,CO-ANT,"Urbanización Atalaya de San Jorge, Envigado, A...",L6664911,P,6.161704,...,NaN,Hypopyrrhus pyrohypogaster,0,False,Vulnerable,0.483333,0.641379,17.016667,2019.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50313,G7638691,2022-01-27 12:01:04.274946,Colombia,CO,Antioquia,CO-ANT,Límites Barbosa-Don Matías,L17261225,P,6.506407,...,NaN,Hypopyrrhus pyrohypogaster,0,False,Vulnerable,0.700000,0.842857,10.566667,2021.0,364.0
50314,G7638693,2022-01-05 13:11:01.131133,Colombia,CO,Antioquia,CO-ANT,Autopista al río Magdalena Popalito,L17261280,P,6.491236,...,NaN,Hypopyrrhus pyrohypogaster,0,False,Vulnerable,0.366667,1.772727,11.733333,2021.0,364.0
50315,G7614303,2023-10-24 04:02:13.955611,Colombia,CO,Antioquia,CO-ANT,Jardin Cock-of-the-Rock Lek,L2719023,H,5.596620,...,NaN,Hypopyrrhus pyrohypogaster,0,False,Vulnerable,0.933333,0.546429,16.950000,2021.0,201.0
50316,G7617030,2021-12-31 19:27:42.369418,Colombia,CO,Antioquia,CO-ANT,O2 Reserve,L17269191,P,6.583066,...,NaN,Hypopyrrhus pyrohypogaster,0,False,Vulnerable,6.000000,1.333333,14.083333,2021.0,362.0


In [65]:
# Variables clasificadas por el tipo de dato equivalente entre Python y R

char_attributes = ["checklist_id","country","country_code","state","state_code",
                   "locality","locality_id","locality_type","observer_id",
                   "sampling_event_identifier","protocol_type","protocol_code",
                   "project_code","group_identifier","trip_comments","scientific_name",
                   "time_observations_started"]

float_attributes= ["latitude","longitude","effort_distance_km",
                   "effort_hours","effort_speed_kmph","hours_of_day","year",
                   "day_of_year"]

int_attributes  = ["duration_minutes","number_observers","observation_count"]

bool_attributes = ["all_species_reported","species_observed"]

In [66]:
# Conversion a tipos de datos string (cadenas)
df_vu[char_attributes] = df_vu[char_attributes].astype(str)

# Conversion a tipos de datos float (flotantes)
df_vu[float_attributes] = df_vu[float_attributes].astype(float)

# Conversion a tipos de datos integer (enteros)
df_vu[int_attributes] = df_vu[int_attributes].astype(int)

# Conversion a tipos de datos boolean (booleanos)
df_vu[bool_attributes] = df_vu[bool_attributes].astype(bool)

In [67]:
# De DataFrame a GeoDataFrame
geometry = [Point(xy) for xy in zip(df_vu['longitude'], df_vu['latitude'])]
gdf_df_vu = GeoDataFrame(df_vu, crs='WGS84', geometry=geometry)

In [68]:
# Selección de Variables de interés a partir del GeoDataFrame de avistamientos
gdf_df_vu = gdf_df_vu[["scientific_name", "year", "latitude","longitude",
                       "observation_count", "geometry","effort_hours",'effort_speed_kmph',
                       'effort_distance_km', 'number_observers']]

In [69]:
# Se transforman los datos a una representacion basada en Unidades Estándar
gdf_df_vu = gdf_df_vu.to_crs(epsg = 5382)

## Shape de Departamentos de Colombia

In [70]:
# # Carga de Shape complementario
dir_Drive_shp = "/content/drive/MyDrive/Proyecto aplicado /Departamentos_Junio_2024_shp/Departamento.shp"

# Carga de Shape complementario: JUAN
# dir_Drive_shp = "/content/drive/MyDrive/research/codes/Proyecto aplicado /Departamentos_Junio_2024_shp/Departamento.shp"

states = gpd.read_file(dir_Drive_shp, use_arrow=True)

# Conversion al Sistema de Referencia de Coordendas
states = states.to_crs(crs=gdf_df_vu.crs)

# Region de Interés
roi = states[states['DeNombre'] == 'Antioquia']

# Reproyección
roi = roi.to_crs(crs=gdf_df_vu.crs)

## Variables exógenas

In [71]:
# Ruta de los datos
ruta_var_exogenas = "/content/drive/MyDrive/Proyecto aplicado /Variables Bioclimaticas/"

# Ruta de los datos Juan
# ruta_var_exogenas = "/content/drive/MyDrive/research/codes/Proyecto aplicado /Variables Bioclimaticas/"

# CRS de los rasters de WorldClim
wc21_crs = rio.open(ruta_var_exogenas + "Elevation_Output_ant/elevation_1.tif").crs

In [72]:
# Carga del raster
mapbiomas_ant = rxr.open_rasterio(ruta_var_exogenas + "MapBiomas_Output_ant/mapbiomas_ant_2014.tif", masked = True).squeeze()
mb_crs = rio.open(ruta_var_exogenas + "MapBiomas_Output_ant/mapbiomas_ant_2014.tif").crs

## Código principal

In [73]:
# Se grafican los datos
fig, axs = plt.subplots(1, 1, figsize=(10, 10), sharey=True, sharex=True)

gdf_df_vu.plot(ax=axs,c='b',markersize=8)
roi.plot(ax=axs,facecolor='none',edgecolor='k')
cx.add_basemap(ax=axs,crs=gdf_df_vu.crs,attribution_size=3)

axs.add_artist(ScaleBar(1))

plt.title("Registros Antioquia Hypopyrrhus pyrohypogaster", fontsize=22)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

# Guardar la figura en formato PNG con calidad 600 dpi y fondo transparente
fig.savefig(ruta_figs + 'fig_21_distribucion_espacial.png', dpi=600, bbox_inches='tight', format='png', transparent=True)

# Mostrar la figura
plt.show()

# Cerrar la figura después de guardarla
plt.close(fig)

Output hidden; open in https://colab.research.google.com to view.

In [74]:
# A partir del shape de la Región de Interés se genera una malla hexagonal
roi_hexagrid = ghg.make_grid_from_gdf(roi, R=2000)

In [75]:
# Recorte del shape
roi_hexagrid = gpd.clip(roi_hexagrid,roi)

In [77]:
# Gráfica de la representacion hexagonal
fig, axs = plt.subplots(1, 1, figsize=(10, 10), sharey=True, sharex=True)

roi_hexagrid.plot(ax=axs,facecolor='black',linewidth=1.5)
gdf_df_vu.plot(ax=axs,c='b',markersize=8)
cx.add_basemap(ax=axs,crs=roi_hexagrid.crs,attribution_size=3)

axs.add_artist(ScaleBar(1))

plt.suptitle("Representacion de malla hexagonal", fontsize=22, y=0.92)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

# Guardar la figura en formato PNG con calidad 600 dpi y fondo transparente
fig.savefig(ruta_figs + 'fig_21_malla.png', dpi=600, bbox_inches='tight', format='png', transparent=True)

# Mostrar la figura
plt.show()

# Cerrar la figura después de guardarla
plt.close(fig)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Se une la información de la malla hexagonal a los datos
malla_avist_vu = gpd.sjoin(roi_hexagrid, gdf_df_vu, how="inner", predicate="contains")

In [ ]:
malla_avist_vu

,cell_id,geometry,index_right,scientific_name,year,latitude,longitude,observation_count,effort_hours,effort_speed_kmph,effort_distance_km,number_observers
572,"-516,6144","POLYGON ((-1549000 10643452.213, -1547000 1064...",13277,Hypopyrrhus pyrohypogaster,2022.0,5.523397,-75.188650,0,2.166667,0.364615,0.79,1
571,"-517,6145","POLYGON ((-1552000 10641720.162, -1553000 1064...",1922,Hypopyrrhus pyrohypogaster,2020.0,5.526343,-75.197102,0,1.000000,1.500000,1.50,1
571,"-517,6145","POLYGON ((-1552000 10641720.162, -1553000 1064...",13287,Hypopyrrhus pyrohypogaster,2022.0,5.531066,-75.191840,0,4.950000,0.581818,2.88,1
571,"-517,6145","POLYGON ((-1552000 10641720.162, -1553000 1064...",13286,Hypopyrrhus pyrohypogaster,2022.0,5.531066,-75.191840,0,4.716667,0.358304,1.69,1
699,"-514,6146","POLYGON ((-1543000 10643452.213, -1544000 1064...",8708,Hypopyrrhus pyrohypogaster,2021.0,5.540416,-75.133521,0,1.266667,1.121053,1.42,1
...,...,...,...,...,...,...,...,...,...,...,...,...
14779,"-559,6373","POLYGON ((-1675000 11038359.797, -1676000 1103...",3919,Hypopyrrhus pyrohypogaster,2020.0,8.860646,-76.420463,0,1.766667,1.415094,2.50,1
14779,"-559,6373","POLYGON ((-1675000 11038359.797, -1676000 1103...",3917,Hypopyrrhus pyrohypogaster,2020.0,8.861374,-76.419811,0,3.916667,1.917447,7.51,1
14779,"-559,6373","POLYGON ((-1675000 11038359.797, -1676000 1103...",35982,Hypopyrrhus pyrohypogaster,2019.0,8.861754,-76.418002,0,2.150000,0.000000,0.00,1
14779,"-559,6373","POLYGON ((-1675000 11038359.797, -1676000 1103...",46227,Hypopyrrhus pyrohypogaster,2020.0,8.863471,-76.418041,0,2.000000,0.015000,0.03,2


In [ ]:
# Conteo de variables de esfuerzo y avistamientos por hexagono
variables = ["effort_speed_kmph", 'effort_hours', 'effort_distance_km',
             'number_observers']

malla_avist_vu_ev = generate_hex_ab_counts(malla_avist_vu,roi_hexagrid,'observation_count')

for var in variables:
    gdf_temp = generate_hex_ab_counts(malla_avist_vu,roi_hexagrid,var)
    malla_avist_vu_ev = pd.merge(malla_avist_vu_ev, gdf_temp.drop(columns='geometry'),
                                    how= "outer", on= "cell_id")

In [ ]:
# Malla de avistamientos
malla_avist_vu_ev.fillna(0, inplace= True)

In [ ]:
# Se crea la columna de etiquetas
malla_avist_vu_ev['label'] = malla_avist_vu_ev['observation_count'].apply(lambda x: 1 if x > 0 else 0)

# A manera de visualización se prueba si se asignaron correctamente las etiquetas
malla_avist_vu_ev[['label','observation_count']][malla_avist_vu_ev['observation_count'] != 0]

,label,observation_count
1386,1,4.0
1543,1,2.0
1943,1,5.0
1963,1,9.0
2033,1,5.0
...,...,...
5301,1,8.0
5302,1,30.0
5303,1,24.0
5353,1,2.0


In [ ]:
# Se aplica función personalizada para imputar espacialmente los datos a partir
# del GeoDataFrame de avistamiento

# Rutas de las variables exógenas: WorldClim + Elevación
rutas_vars_exogenas = [ruta_var_exogenas + "Elevation_Output_ant/elevation_1.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_1.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_2.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_3.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_4.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_5.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_6.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_7.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_8.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_9.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_10.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_11.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_12.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_13.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_14.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_15.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_16.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_17.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_18.tif",
                       ruta_var_exogenas + "WorldClim_Output_ant/worldclim_bio_19.tif"
                       ]

# Lista de nombres de las nuevas variables
var_list = ['elevation','anual_mean_t', 'mean_diurnal_range',
            'isothermality', 't_seasonality', 'max_t_warmest_month',
            'min_t_coldest_month', 't_annual_range', 'mean_t_wettest_q',
            'mean_t_driest_q','mean_t_warmest_q','mean_t_coldest_q','annual_p',
            'p_wettest_m','p_driest_m', 'p_seasonality', 'p_wettest_q',
            'p_driest_q', 'p_warmest_q', 'p_coldest_q']

# Se imputan las nuevas variables al GeoDataFrame
malla_avist_vu_ev_wc = data_imputation(malla_avist_vu_ev,
        rutas_vars_exogenas,
        wc21_crs,
        'label',
        var_list)

In [ ]:
# Se descartan los valores faltantes y/o nulos
malla_avist_vu_ev_wc.dropna(inplace=True)

In [ ]:
# Se aplica función personalizada para imputar espacialmente los datos a partir
# del GeoDataFrame de avistamiento

# Rutas de las variables exógenas: MapBiomas
rutas_vars_exogenas = [ruta_var_exogenas + "MapBiomas_Output_ant/mapbiomas_ant_2014.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ant/mapbiomas_ant_2015.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ant/mapbiomas_ant_2016.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ant/mapbiomas_ant_2017.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ant/mapbiomas_ant_2018.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ant/mapbiomas_ant_2019.tif",
                       ruta_var_exogenas + "MapBiomas_Output_ant/mapbiomas_ant_2020.tif"]

# Lista de nombres de las nuevas variables
var_list = ['lulc_2014','lulc_2015','lulc_2016','lulc_2017','lulc_2018','lulc_2019','lulc_2020']

# Se imputan las nuevas variables al GeoDataFrame
malla_avist_vu_mb = data_imputation(malla_avist_vu_ev,
        rutas_vars_exogenas,
        mb_crs,
        'label',
        var_list)

In [ ]:
# Le descartamos a este GeoDataGrame las variables de esfuerzo, el conteo de
# observaciones y las etiquetas ya que están en el otro GeoDataFrame al que vamos
# a unirlo
malla_avist_vu_mb.drop(columns=["effort_speed_kmph", 'effort_hours', 'effort_distance_km',
             'number_observers', 'observation_count','label'], inplace=True)
malla_avist_vu_mb

,cell_id,geometry,lulc_2014,lulc_2015,lulc_2016,lulc_2017,lulc_2018,lulc_2019,lulc_2020
0,"-464,6240",POINT (-73.8933 7.00063),21.0,21.0,21.0,21.0,21.0,21.0,21.0
1,"-464,6242",POINT (-73.89155 7.02506),21.0,21.0,21.0,21.0,21.0,21.0,21.0
2,"-464,6244",POINT (-73.89433 7.05521),21.0,21.0,21.0,21.0,21.0,21.0,21.0
3,"-464,6246",POINT (-73.89736 7.07805),33.0,33.0,33.0,33.0,33.0,33.0,33.0
4,"-465,6239",POINT (-73.91399 6.98307),21.0,21.0,21.0,21.0,21.0,21.0,21.0
...,...,...,...,...,...,...,...,...,...
7117,"-587,6303",POINT (-77.09574 7.79475),3.0,3.0,3.0,3.0,3.0,3.0,3.0
7118,"-587,6305",POINT (-77.09588 7.81951),11.0,11.0,11.0,11.0,11.0,11.0,11.0
7119,"-588,6302",POINT (-77.11322 7.78612),6.0,6.0,6.0,6.0,6.0,6.0,6.0
7120,"-588,6304",POINT (-77.11676 7.80881),21.0,3.0,3.0,3.0,3.0,3.0,3.0


In [ ]:
# Se unen los datos de WorldClim + Elevacion con los de MapBiomas para el departamento
malla_avist_vu_ev_f = gpd.sjoin(malla_avist_vu_ev_wc, malla_avist_vu_mb,on_attribute='cell_id')

In [ ]:
malla_avist_vu_ev_f

,cell_id,observation_count,geometry,effort_speed_kmph,effort_hours,effort_distance_km,number_observers,label,elevation,anual_mean_t,...,p_warmest_q,p_coldest_q,index_right,lulc_2014,lulc_2015,lulc_2016,lulc_2017,lulc_2018,lulc_2019,lulc_2020
0,"-464,6240",0.0,POINT (-73.8933 7.00063),0.000000,0.000000,0.00,0.0,0,71.0,28.112501,...,281.0,967.0,0,21.0,21.0,21.0,21.0,21.0,21.0,21.0
1,"-464,6242",0.0,POINT (-73.89155 7.02506),0.000000,0.000000,0.00,0.0,0,71.0,28.112501,...,281.0,967.0,1,21.0,21.0,21.0,21.0,21.0,21.0,21.0
2,"-464,6244",0.0,POINT (-73.89433 7.05521),1.250000,4.183333,5.00,3.0,0,71.0,28.192833,...,283.0,968.0,2,21.0,21.0,21.0,21.0,21.0,21.0,21.0
3,"-464,6246",0.0,POINT (-73.89736 7.07805),0.000000,0.000000,0.00,0.0,0,71.0,28.192833,...,283.0,968.0,3,33.0,33.0,33.0,33.0,33.0,33.0,33.0
4,"-465,6239",0.0,POINT (-73.91399 6.98307),8.185726,8.633333,13.99,11.0,0,75.0,28.035999,...,264.0,926.0,4,21.0,21.0,21.0,21.0,21.0,21.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7117,"-587,6303",0.0,POINT (-77.09574 7.79475),0.000000,0.000000,0.00,0.0,0,4.0,27.132500,...,502.0,828.0,7117,3.0,3.0,3.0,3.0,3.0,3.0,3.0
7118,"-587,6305",0.0,POINT (-77.09588 7.81951),0.000000,0.000000,0.00,0.0,0,4.0,27.132500,...,502.0,828.0,7118,11.0,11.0,11.0,11.0,11.0,11.0,11.0
7119,"-588,6302",0.0,POINT (-77.11322 7.78612),1.140000,0.166667,0.19,3.0,0,3.0,27.167334,...,517.0,864.0,7119,6.0,6.0,6.0,6.0,6.0,6.0,6.0
7120,"-588,6304",0.0,POINT (-77.11676 7.80881),11.053846,0.383333,2.35,6.0,0,4.0,27.132500,...,502.0,828.0,7120,21.0,3.0,3.0,3.0,3.0,3.0,3.0


In [ ]:
# Se almacena el shapefile.
malla_avist_vu_ev_f.to_file('/content/drive/MyDrive/Proyecto aplicado /datos_unidos_avist_wc_mb_ant_vu/datos_unidos_avist_wc_mb_ant_vu.shp')

# Se almacena el shapefile Juan
# malla_avist_vu_ev_f.to_file('/content/drive/MyDrive/research/codes/Proyecto aplicado /datos_unidos_avist_wc_mb_ris_en/datos_unidos_avist_wc_mb_ris_en.shp')

<ipython-input-115-6509cd507a91>:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  malla_avist_vu_ev_f.to_file('/content/drive/MyDrive/Proyecto aplicado /datos_unidos_avist_wc_mb_ant_vu/datos_unidos_avist_wc_mb_ant_vu.shp')
/usr/local/lib/python3.10/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'observation_count' to 'observatio'
  ogr_write(
/usr/local/lib/python3.10/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'effort_speed_kmph' to 'effort_spe'
  ogr_write(
/usr/local/lib/python3.10/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'effort_hours' to 'effort_hou'
  ogr_write(
/usr/local/lib/python3.10/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'effort_distance_km' to 'effort_dis'
  ogr_write(
/usr/local/lib/python3.10/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/launde